### 구현 방식
1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면(인풋이 길면) 답변 생성이 오래걸린다.
3. (쪼갠 문서를) 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때 벡터 DB에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,    # 문서를 쪼갤 때 하나의 chunk가 가지는 token 수
    chunk_overlap=200   # 쪼갠 내용에서 겹치는 정도
)
loader = Docx2txtLoader("./tax_with_markdown.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [3]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = "tax-markdown-index" #Pinecone Console 사이트에서 만든 index 이름
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)


# 데이터를 추가할 때는 --> from_documents()
# database = PineconeVectorStore.from_documents(
#     document_list,
#     embedding,
#     index_name = index_name
# )

# 데이터를 추가한 이후에는 --> from_existing_index() 사용
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

c:\Users\terious\Desktop\RAG\env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
query = "연봉 5,000만 원인 거주자의 소득세는 얼마인가요?"

In [5]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(id='e1187097-caa7-4034-98a9-5bad50bf8813', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

In [6]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [8]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt" : prompt}
)
retriever

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000002754354EED0>, search_kwargs={'k': 4})

In [9]:
ai_message = qa_chain({"query" : query})
ai_message

C:\Users\terious\AppData\Local\Temp\ipykernel_8132\1666302034.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query" : query})


{'query': '연봉 5,000만 원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5,000만 원인 거주자의 소득세는 624만 원입니다. 이 금액은 5,000만 원 이하의 종합소득 과세표준에 해당하므로, 84만 원에 1,400만 원을 초과하는 금액의 15%를 더하여 계산됩니다.'}